In [1]:
import itertools
import argparse
import numpy as np
import pandas as pd
import sklearn
import random
import sklearn.metrics
import time
import os
import sys
import json
import gc
import json
import hashlib
import tempfile
import keras.optimizers
import pickle
import datetime
import cv2
import math
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Reshape
import ast
import h5py
from keras.utils import np_utils
from itertools import tee
import keras.backend as K
import tensorflow as tf
from copy import deepcopy
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from IPython.display import display, clear_output
import PIL.Image
from cStringIO import StringIO
import IPython.display
import matplotlib.pyplot as plt

%run util.py

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.25
keras.backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


ModuleNotFoundError: No module named 'cStringIO'

In [ ]:
FRAMES_PER_SEC = 10 # frame rat e we decode the video
FULL_RESOL = (720, 1280) # default 720p

def str2ilst(s):
    return [int(x) for x in s.split(',')]

def add_json_file(key, val, fname):
    res = {}
    if os.path.exists(fname):
        with open(fname) as f:
            res = json.load(f)
    res[key] = val
    with open(fname, 'w+') as out_f:
        json.dump(res, out_f)

def already_saved(key, fname):
    if os.path.exists(fname):
        with open(fname) as f:
            res = json.load(f)
            if key in res: return True
    return False

def get_score_fname(cur_cfg):
    scores_dir = os.path.join(CUR_RES_ROOT_PATH, 'scores')
    if not os.path.exists(scores_dir):
        os.mkdir(scores_dir)
    cfg_str = json.dumps(cur_cfg)
    cfg_md5 = hashlib.md5(cfg_str).hexdigest()
    cfg_fname = os.path.join(scores_dir, cfg_md5)
    return cfg_fname

def read_test_data_ids(cur_cfg, wnd_id, candidate_ids):
    video = cur_cfg['video']
    obj = cur_cfg['OBJECT']
    testids_dir = os.path.join(CUR_RES_ROOT_PATH, 'testids')
    if not os.path.exists(testids_dir):
        os.mkdir(testids_dir)
    testid_fname = os.path.join(testids_dir, video + '-' + obj + '-' + str(wnd_id))
    if not os.path.exists(testid_fname):
        print 'Create test id file...'
        with open(testid_fname, 'wb') as f:
            pickle.dump(candidate_ids, f)
    with open(testid_fname, 'rb') as f:
        return pickle.load(f)

def read_input(obj_ids, input_cfg, wnd_size_lst, img_sub_dirs,
               img_sub_nums, cur_cfg, max_resol=(100, 100)):
    print 'reading input for config:', input_cfg
    train_num = input_cfg[0]
    test_num = input_cfg[1]
    wnd_num = input_cfg[2]
    train_pos_ratio = input_cfg[3]
    crop = str2ilst(input_cfg[4])
    train_num_wnd = train_num / wnd_num
    test_num_wnd = test_num / wnd_num
    train_ids = []
    test_ids = []
    for i in range(wnd_num):
        ids_wnd = range(sum(wnd_size_lst[:i]), sum(wnd_size_lst[:i+1]), 1)
        test_ids_candidate = random.sample(ids_wnd, test_num_wnd)
        test_ids_wnd = read_test_data_ids(cur_cfg, i, test_ids_candidate)
        if len(test_ids_wnd) != test_num_wnd:
            print 'test id number not matched!!!!!'
            sys.exit()
        
        temp_ids_wnd = set(ids_wnd) - set(test_ids_wnd)
        train_ids_pos_wnd_total = set(temp_ids_wnd) & set(obj_ids)
        real_pos_ratio = float(len(train_ids_pos_wnd_total)) / len(temp_ids_wnd)
        if train_pos_ratio < real_pos_ratio:
            train_ids_wnd = random.sample(temp_ids_wnd, train_num_wnd)
        else:
            pos_num_needed = int(train_num_wnd * train_pos_ratio)
            pos_num_needed = min(
                pos_num_needed,
                int(real_pos_ratio * train_num_wnd * 30), # at least 30 frames
                len(train_ids_pos_wnd_total) / 2)
            train_ids_pos_wnd = random.sample(
                train_ids_pos_wnd_total, pos_num_needed)
            train_ids_neg_wnd = random.sample(
                temp_ids_wnd - train_ids_pos_wnd_total,
                train_num_wnd - pos_num_needed)
            train_ids_wnd = train_ids_pos_wnd + train_ids_neg_wnd
            random.shuffle(train_ids_wnd)
        train_ids += train_ids_wnd
        test_ids += test_ids_wnd
    X_train = get_frames_from_images(
        train_ids, img_sub_dirs, img_sub_nums, resol=max_resol, crop=crop)
    Y_train = get_labels(obj_ids, train_ids)
    X_test = get_frames_from_images(
        test_ids, img_sub_dirs, img_sub_nums, resol=max_resol, crop=crop)
    Y_test = get_labels(obj_ids, test_ids)
    return X_train, Y_train, X_test, Y_test

def gen_model(resol, model_param, pre_model=None, model_type='AlexNet'):
    if model_type == 'AlexNet':
        param = ((resol[0], resol[1], 3), 2, model_param[0], model_param[1], model_param[2])
        model = generate_conv_net_base(*param, lr_mult=0.001)
    elif model_type == 'MobileNet':
        model = keras.applications.mobilenet.MobileNet(
            input_shape=(resol[0], resol[1], 3),
            alpha=0.25, depth_multiplier=1, dropout=1e-3,
            include_top=True, weights=None, classes=2)
        model.compile(loss=get_loss(),
                  optimizer=get_optimizer(lr_mult=0.001),
                  metrics=computed_metrics)
    else:
        print ('No such model supported: ' + model_type)
    if pre_model != None:
        model.set_weights(pre_model.get_weights())
    return model

def run_models(X_train, Y_train, X_test, Y_test, video_cfg, cur_cfg):
    nb_epoch=10
    wnd_num = cur_cfg['input_cfg'][2]
    out_file = os.path.join(CUR_RES_ROOT_PATH, cur_cfg['video'] + '.json')
    resol = cur_cfg['resol']
#     X_train_total = np.concatenate(X_train_wnds, axis=0)
#     Y_train_total = np.concatenate(Y_train_wnds, axis=0)
#     X_test_total = np.concatenate(X_test_wnds, axis=0)
#     Y_test_total = np.concatenate(Y_test_wnds, axis=0)
    
    mean = np.mean(X_train, axis=0)
    save_mean(cur_cfg, mean)
    X_train = X_train - mean
    X_test = X_test - mean
    
    max_conv_num = int(math.log(min(resol), 2))
    cfgs = []
    for conv_num in range(max_conv_num):
        for cfg in video_cfg['model']:
            cfgs.append(cfg + ',' + str(conv_num + 1))
    for model_cfg in cfgs:
        print 'running cfg: ' + model_cfg
        cur_cfg['model'] = model_cfg
        param = str2ilst(model_cfg)
        K.clear_session()
        
        cur_cfg['generic'] = True
        if already_saved(json.dumps(cur_cfg), out_file):
            print 'skip...'
            continue
        generic_model = gen_model(resol, param)
        train_model(generic_model, X_train, Y_train, batch_size=32,
                    nb_epoch=20, save_path=get_model_fname(cur_cfg))
        scores = test_model(generic_model, X_test, Y_test)
        add_json_file(json.dumps(cur_cfg), str(datetime.datetime.now()), out_file)
        with open(get_score_fname(cur_cfg), 'w+') as f:
            np.save(f, scores)
        
        cur_cfg['generic'] = False
        total_scores = []
        wnd_size_train = X_train.shape[0] / wnd_num
        wnd_size_test = X_test.shape[0] / wnd_num
        for i in range(wnd_num):
            start_train = i * wnd_size_train
            end_train = (i + 1) * wnd_size_train
            start_test = i * wnd_size_test
            end_test = (i + 1) * wnd_size_test
            model = gen_model(resol, param, pre_model=generic_model)
            train_model(model, X_train[start_train:end_train], Y_train[start_train:end_train],
                        batch_size=32, nb_epoch=10, save_path=get_model_fname(cur_cfg) + '_' + str(i))
            res = test_model(model, X_test[start_test:end_test], Y_test[start_test:end_test])
            total_scores.append(res)
        add_json_file(json.dumps(cur_cfg), str(datetime.datetime.now()), out_file)
        total_scores = np.concatenate(total_scores, axis=0)
        with open(get_score_fname(cur_cfg), 'w+') as f:
            np.save(f, total_scores)
        
        
def run_once(video_name, video_cfg):
    # iterate input formats
    video_clip_num = video_cfg['VIDEO_CLIP_NUM']
    SMOOTH_WINDOW = FRAMES_PER_SEC * video_cfg['SMOOTH_SEC'] # for smooth the yolo output
    input_cfgs = list(itertools.product(
        video_cfg['train_num'],
        video_cfg['test_num'],
        video_cfg['wnd_num'],
        video_cfg['train_pos_ratio'],
        video_cfg['crop']
    ))
    sub_videos = [video_name + '-' + str(p + 1) + '_10FPS' for p in range(video_clip_num)]
    img_sub_dirs = [os.path.join(JPG_ROOT_PATH, v) for v in sub_videos]
    img_sub_nums = [len(os.listdir(d)) for d in img_sub_dirs]
    total_frame_num = sum(img_sub_nums)
    csv_sub_files = [os.path.join(CSV_ROOT_PATH, d + '.csv') for d in sub_videos]
    cur_cfg = {'video': video_name, 'smooth_window': SMOOTH_WINDOW}
    for OBJECT in video_cfg['OBJECT']:
        cur_cfg['OBJECT'] = OBJECT
        for input_cfg in input_cfgs:
            cur_cfg['input_cfg'] = input_cfg
            obj_ids, _ = get_csv_samples_many(
                csv_sub_files, img_sub_nums, OBJECT, total_frame_num,
                WINDOW=SMOOTH_WINDOW, crop=str2ilst(input_cfg[4]))
#             obj_ids_test, _ = get_csv_samples_many(
#                 csv_sub_files, img_sub_nums, OBJECT, total_frame_num,
#                 WINDOW=SMOOTH_WINDOW, crop=None)
            resol_lst = [str2ilst(r) for r in video_cfg['resol']]
            max_resol = (max([r[0] for r in resol_lst]), max([r[1] for r in resol_lst]))
            X_train, Y_train, X_test, Y_test = read_input(
                obj_ids, input_cfg, img_sub_nums, img_sub_dirs,
                img_sub_nums, cur_cfg, max_resol=max_resol)
            for resol in resol_lst:
                cur_cfg['resol'] = resol
                if resol[0] != max_resol[0] or resol[1] != max_resol[1]:
                    resized_X_train = resize_images(X_train, resol, dtype='float32')
                    resized_X_test = resize_images(X_test, resol, dtype='float32')
                else:
                    resized_X_train = X_train
                    resized_X_test = X_test
                run_models(resized_X_train, Y_train, resized_X_test, Y_test, video_cfg, cur_cfg)
                resized_X_train = None
                resized_X_test = None
                gc.collect()
            X_train = None
            X_test = None
            Y_train = None
            Y_test = None

In [ ]:
# automate experiments
with open('exp_params.cfg') as f:
    cfg = json.load(f)

run_video = 'venice-canal'
CUR_RES_ROOT_PATH = os.path.join(RES_ROOT_PATH, run_video)
if not os.path.exists(CUR_RES_ROOT_PATH):
    os.mkdir(CUR_RES_ROOT_PATH)
for video in cfg.keys():
    if video != run_video: continue
    run_once(video, cfg[video])
    gc.collect()

## test background subtraction

In [ ]:
# test the background subtraction
video_name = 'zoo'
obj = 'person'
video_clip_num = 8

%run util.py

sub_videos = [video_name + '-' + str(p + 1) + '_10FPS' for p in range(video_clip_num)]
img_sub_dirs = [os.path.join(JPG_ROOT_PATH, v) for v in sub_videos]
img_sub_nums = [len(os.listdir(d)) for d in img_sub_dirs]
total_frame_num = sum(img_sub_nums)

csv_sub_files = [os.path.join(CSV_ROOT_PATH, d + '.csv') for d in sub_videos]
obj_ids, _ = get_csv_samples_many(
    csv_sub_files, img_sub_nums, obj, total_frame_num,
    WINDOW=50, crop=None)


fgbg_dict = {}
out_dir = '/host/4TB_hybridvs_data/YOLO-RES-720P/fgbg/'
for i in range(video_clip_num):
    out_fname = os.path.join(out_dir, video_name + '-' + str(i + 1))
    with open(out_fname, 'rb') as f:
        base = sum(img_sub_nums[:i])
        res = (pickle.load(f))
        for jpg in res:
            idx = int(jpg[:-4]) + base
            fgbg_dict[idx] = res[jpg]

In [ ]:
def get_big_box(matches):
    min_x = min(m[0] for m in matches)
    min_y = min(m[1] for m in matches)
    max_x = max(m[2] for m in matches)
    max_y = max(m[3] for m in matches)
    return (min_x, min_y, max_x, max_y)

def contain_rect(br, sr):
    return br[0] <= sr[0] and br[1] <= sr[1] and br[2] >= sr[2] and br[3] >= sr[3]

def convert(crop):
    return (crop[1], crop[0], crop[1] + crop[3], crop[0] + crop[2])

def get_frames_from_images(frameset, img_dirs, img_nums, fgmask_dict=None,
                           resol=(-1, -1), crop=(-1, -1, -1, -1), dtype='float32'):
    def get_image_path(idx):
        for i in range(len(img_dirs)):
            if sum(img_nums[:i + 1]) > idx:
                return os.path.join(img_dirs[i], str(idx - sum(img_nums[:i]) + 1).zfill(7) + '.jpg')
    assert len(img_dirs) == len(img_nums), 'image subdir numbers not consistent!'
#     print ('reading images... %d from %s...') % (len(frameset), img_dirs[0])
    if resol[0] > 0:
        frames = np.zeros(tuple([len(frameset)] + list(resol) + [3]), dtype=dtype )
    else:
        test_frame = cv2.imread(os.path.join(img_dirs[0], '0000001.jpg'))
        frames = np.zeros(tuple([len(frameset)] + list(test_frame.shape)), dtype=dtype )
    for i in range(len(frameset)):
        img_path = get_image_path(frameset[i])
        frame = cv2.imread(img_path)
        tmp_crop = deepcopy(crop)
        if fgmask_dict is not None:
            tmp_idx = frameset[i]
            while tmp_idx not in fgmask_dict:
                tmp_idx -= 1
            boxes = fgmask_dict[tmp_idx]
            boxes = [b for b in boxes if b[2] > 40 and b[3] > 40]
            boxes = [convert(b) for b in boxes]
            if tmp_crop[0] > 0:
                boxes = [b for b in boxes if contain_rect(tmp_crop, b)]
            if len(boxes) > 0:
                tmp_crop = get_big_box(boxes)
        if tmp_crop[0] > 0:
            frame = frame[tmp_crop[0]:tmp_crop[2],tmp_crop[1]:tmp_crop[3],:]
        if resol[0] > 0:
            frame = cv2.resize(frame, (resol[1], resol[0]), interpolation=cv2.INTER_NEAREST)
        frames[i, :] = frame

    if dtype == 'float32':
        frames /= 255.0

    return frames

max_num = max(fgbg_dict.keys())
train_ids = random.sample(range(max_num), 24000)
test_ids = random.sample(range(max_num), 20000)

X_train = get_frames_from_images(
    train_ids, img_sub_dirs, img_sub_nums, fgmask_dict=fgbg_dict,
    resol=(50,50), crop=(-1,-1,-1,-1), dtype='float32')
Y_train = get_labels(obj_ids, train_ids)
X_test = get_frames_from_images(
    test_ids, img_sub_dirs, img_sub_nums, fgmask_dict=fgbg_dict,
    resol=(50,50), crop=(-1,-1,-1,-1), dtype='float32')
Y_test = get_labels(obj_ids, test_ids)

In [ ]:
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
    
showarray(X_train[8000,:] * 255)

In [ ]:
def gen_model((r1, r2), nb_dense, nb_filters, nb_layers, pre_model=None):
    param = ((r1, r2, 3), 2, nb_dense, nb_filters, nb_layers)
    model = generate_conv_net_base(*param, lr_mult=0.001)
    return model

def train_test(param, cur_X_train, cur_Y_train, cur_X_test, cur_Y_test):
    K.clear_session()
    model = gen_model(*param)
    train_model(model, cur_X_train, cur_Y_train, batch_size=32, nb_epoch=20)
    scores = test_model(model, cur_X_test, cur_Y_test, batch_size=128)
    return cal_scores(scores), cal_score_filter(scores)


acc1 = train_test(((50,50),32,16,5), X_train, Y_train, X_test, Y_test)

In [ ]:
print acc1[1]

In [ ]:
X_train_2 = get_frames_from_images(
    train_ids, img_sub_dirs, img_sub_nums, fgmask_dict=None,
    resol=(50,50), crop=(-1,-1,-1,-1), dtype='float32')
X_test_2 = get_frames_from_images(
    test_ids, img_sub_dirs, img_sub_nums, fgmask_dict=None,
    resol=(50,50), crop=(-1,-1,-1,-1), dtype='float32')

In [ ]:
acc2 = train_test(((50,50),32,16,5), X_train_2, Y_train, X_test_2, Y_test)

In [ ]:
print acc2[1]

In [ ]:
# def read_input(obj_ids, obj_ids_test, input_cfg, wnd_size_lst, img_sub_dirs,
#                img_sub_nums, cur_cfg, max_resol=(100, 100)):
#     print 'reading input for config:', input_cfg
#     train_num = input_cfg[0]
#     test_num = input_cfg[1]
#     wnd_num = input_cfg[2]
#     train_pos_ratio = input_cfg[3]
#     crop = str2ilst(input_cfg[4])
#     train_num_wnd = train_num / wnd_num
#     test_num_wnd = test_num / wnd_num
#     frame_ids_wnd = []
#     train_ids = []
#     X_train_wnds = []
#     Y_train_wnds = []
#     test_ids = []
#     X_test_wnds = []
#     Y_test_wnds = []
#     for i in range(wnd_num):
#         print 'processing time window: ', i
#         ids_wnd = range(sum(wnd_size_lst[:i]), sum(wnd_size_lst[:i+1]), 1)
#         least_pos_test_ids = random.sample(obj_ids, 5)
#         test_ids_candidate = least_pos_test_ids + random.sample(
#             set(ids_wnd) - set(least_pos_test_ids), test_num_wnd - 5)
#         test_ids_wnd = read_test_data_ids(cur_cfg, i, test_ids_candidate)
#         if len(test_ids_wnd) != test_num_wnd:
#             print 'test id number not matched!!!!!'
#             sys.exit()
#         temp_ids_wnd = [x for x in ids_wnd if x not in test_ids_wnd]
#         if train_pos_ratio < 0:
#             train_ids_wnd = random.sample(temp_ids_wnd, train_num_wnd)
#         else:
#             train_ids_pos_wnd_total = [x for x in temp_ids_wnd if x in obj_ids]
#             pos_num_needed = int(train_num_wnd * train_pos_ratio)
#             if len(train_ids_pos_wnd_total) < pos_num_needed:
#                 pos_num_needed = len(train_ids_pos_wnd_total) / 2
#             train_ids_pos_wnd = random.sample(
#                 train_ids_pos_wnd_total, pos_num_needed)
#             train_ids_neg_wnd = random.sample(
#                 [x for x in temp_ids_wnd if x not in obj_ids],
#                 int(train_num_wnd * (1 - train_pos_ratio)))
#             train_ids_wnd = train_ids_pos_wnd + train_ids_neg_wnd
#             random.shuffle(train_ids_wnd)
#         train_ids.append(train_ids_wnd)
#         test_ids.append(test_ids_wnd)
#         X_train_wnds.append(get_frames_from_images(
#             train_ids_wnd, img_sub_dirs, img_sub_nums, resol=max_resol, crop=crop))
#         Y_train_wnds.append(get_labels(obj_ids, train_ids_wnd))
#         X_test_wnds.append(get_frames_from_images(
#             test_ids_wnd, img_sub_dirs, img_sub_nums, resol=max_resol, crop=crop))
#         Y_test_wnds.append(get_labels(obj_ids_test, test_ids_wnd))
#     return X_train_wnds, Y_train_wnds, X_test_wnds, Y_test_wnds


# with open('exp_params.cfg') as f:
#     cfg = json.load(f)

# video_name = 'banff'
# video_cfg = cfg[video_name]

# video_clip_num = video_cfg['VIDEO_CLIP_NUM']
# input_cfgs = list(itertools.product(
#     video_cfg['train_num'],
#     video_cfg['test_num'],
#     video_cfg['wnd_num'],
#     video_cfg['train_pos_ratio'],
#     video_cfg['crop']
# ))
# sub_videos = [video_name + '-' + str(p + 1) for p in range(video_clip_num)]
# img_sub_dirs = [os.path.join(JPG_ROOT_PATH, v) for v in sub_videos]
# img_sub_dirs = [x + '_10FPS' for x in img_sub_dirs]
# img_sub_nums = [len(os.listdir(d)) for d in img_sub_dirs]
# total_frame_num = sum(img_sub_nums)
# csv_sub_files = [os.path.join(CSV_ROOT_PATH, d + '_10FPS.csv') for d in sub_videos]
# cur_cfg = {'video': video_name}
# OBJECT = video_cfg['OBJECT'][0]
# cur_cfg['OBJECT'] = OBJECT
# input_cfg = input_cfgs[0]
# print input_cfg
# cur_cfg['input_cfg'] = input_cfg
# obj_ids, _ = get_csv_samples_many(
#     csv_sub_files, img_sub_nums, OBJECT, total_frame_num,
#     WINDOW=60, crop=str2ilst(input_cfg[4]))
# obj_ids_test, _ = get_csv_samples_many(
#     csv_sub_files, img_sub_nums, OBJECT, total_frame_num,
#     WINDOW=60, crop=None)
# _resol = video_cfg['resol'][0]
# resol = str2ilst(_resol)
# cur_cfg['resol'] = resol
# X_train_wnds, Y_train_wnds, X_test_wnds, Y_test_wnds = read_input(
#     obj_ids, obj_ids_test, input_cfg, img_sub_nums, img_sub_dirs, img_sub_nums, cur_cfg, max_resol=resol)

In [ ]:
# input_cfg2 = (input_cfg[0], input_cfg[1], input_cfg[2], input_cfg[3], "550,300,720,550")
# X_train_wnds2, Y_train_wnds2, X_test_wnds2, Y_test_wnds2 = read_input(
#     obj_ids, obj_ids_test, input_cfg2, img_sub_nums, img_sub_dirs, img_sub_nums, cur_cfg, max_resol=resol)

In [ ]:
# def showarray(a, fmt='png'):
#     a = np.uint8(a)
#     f = StringIO()
#     PIL.Image.fromarray(a).save(f, fmt)
#     IPython.display.display(IPython.display.Image(data=f.getvalue()))

# showarray(X_train_wnds[0][0] * 255)
# showarray(X_train_wnds2[0][0] * 255)

In [ ]:
# nb_epoch=10
# conv_num = 4
# resol = cur_cfg['resol']
# X_train_total = np.concatenate(X_train_wnds, axis=0)
# Y_train_total = np.concatenate(Y_train_wnds, axis=0)
# X_test_total = np.concatenate(X_test_wnds, axis=0)
# Y_test_total = np.concatenate(Y_test_wnds, axis=0)
# mean = np.mean(X_train_total, axis=0)
# X_train_total = X_train_total - mean
# X_test_total = X_test_total - mean
# max_conv_num = int(math.log(min(resol), 2))
# print video_cfg
# model_cfg = video_cfg['model'][0]
# model_cfg = model_cfg + ',' + str(conv_num)
# print 'running cfg: ' + model_cfg
# cur_cfg['model'] = model_cfg
# param = str2ilst(model_cfg)
# K.clear_session()
# generic_model = gen_model(resol, param)
# # generic_model = generate_conv_net_base2()
# train_model(generic_model, X_train_total, Y_train_total, batch_size=64,
#             nb_epoch=nb_epoch)
# scores = test_model(generic_model, X_test_total, Y_test_total)
# print cal_score_filter(scores)
        
# cur_cfg['generic'] = False
# pos_idxs_lst = []
# for i in range(len(X_train_wnds)):
#     model = gen_model(resol, param, pre_model=generic_model)
#     train_model(model, X_train_wnds[i], Y_train_wnds[i], batch_size=32,
#                 nb_epoch=10, save_path=get_model_fname(cur_cfg) + '_' + str(i))
#     accuracy, pos_idxs = test_model(model, X_test_wnds[i], Y_test_wnds[i],
#                                     save_fname=get_score_fname(cur_cfg) + '_' + str(i))
#     pos_idxs_lst.append(pos_idxs)
# formated_acc = cal_accuracy_many(pos_idxs_lst, error_list=[0.5, 0.1, 0.01])
# print formated_acc

In [ ]:
# formated_acc = cal_accuracy(pos_idxs[0], pos_idxs[1], error_list=[0.5, 0.1, 0.01])
# print formated_acc

In [2]:
param = ((100, 100, 3), 2, 64, 32, 6)
model = generate_conv_net_base(*param, lr_mult=0.001)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


<string>:102: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", strides=(1, 1), input_shape=(100, 100,...)`


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<string>:111: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
<string>:111: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3), padding="same")`
<string>:111: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
<string>:111: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(160, (3, 3), padding="same")`
<string>:111: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (3, 3), padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
__________

In [3]:
def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=K.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops  # Prints the "flops" of the model.


# .... Define your model here ....
print(get_flops(model))

Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
1325620


In [9]:
def train_test(param, cur_X_train, cur_Y_train):
    K.clear_session()
    model = gen_model(*param)
    train_model(model, cur_X_train, cur_Y_train, batch_size=32, nb_epoch=20)

def gen_model((r1, r2), nb_dense, nb_filters, nb_layers, pre_model=None):
    param = ((r1, r2, 3), 2, nb_dense, nb_filters, nb_layers)
    model = generate_conv_net_base(*param, lr_mult=0.001)
    return model

X_train = np.random.random((15000,50,50,3))
Y_train = np.random.random((15000,2))
acc1 = train_test(((50,50),64,32,5), X_train, Y_train)

training samples: 7487/15000
Epoch 1/20
15000/15000 [==============================] - 6s 418us/step - loss: 0.7195 - acc: 0.5009 - mean_squared_error: 0.0924
Epoch 2/20
15000/15000 [==============================] - 5s 344us/step - loss: 0.6977 - acc: 0.5022 - mean_squared_error: 0.0840
Epoch 3/20
15000/15000 [==============================] - 5s 343us/step - loss: 0.6971 - acc: 0.4978 - mean_squared_error: 0.0837
Epoch 4/20
15000/15000 [==============================] - 5s 341us/step - loss: 0.6970 - acc: 0.5009 - mean_squared_error: 0.0836
Epoch 5/20
15000/15000 [==============================] - 5s 344us/step - loss: 0.6968 - acc: 0.5015 - mean_squared_error: 0.0836
Epoch 6/20
15000/15000 [==============================] - 5s 341us/step - loss: 0.6968 - acc: 0.4974 - mean_squared_error: 0.0835
Epoch 7/20
15000/15000 [==============================] - 5s 344us/step - loss: 0.6968 - acc: 0.4949 - mean_squared_error: 0.0835
Epoch 8/20
15000/15000 [==============================] - 5s 